In [256]:
import torch
import pandas as pd

In [403]:
features = torch.tensor([[1.0, 2.0], [2.0, 1.0], [-1.0, -2.0]]) 
labels = torch.tensor([1, 0, 0]) 
initial_weights = torch.tensor([0.1, -0.2])
initial_bias = torch.tensor(0.0) 
learning_rate = 0.1 
epochs = 5

def sigmoid_neuron(features, labels, initial_weights, initial_bias, learning_rate, epochs):
    w = initial_weights.clone()
    b = initial_bias.clone()
    losses = []
    eps = 1e-20
    for _ in range(epochs):
        pred = torch.sigmoid(w @ features.T + b)
        loss = -torch.mean(torch.mul(labels, torch.log(pred+eps)) + torch.mul((1-labels), torch.log(1-pred+eps)))
        losses.append(round(loss.item(), 4))

        # sig(z) = 1 / (1 + e^(-z)), z = wx + b
        # L = -(ylog(sig(z)) + (1-y)log(1 - sig(z)))
        # dL/dw = (sig(z) - y)x
        # dL/db = sig(z) - y

        w -= learning_rate * torch.mean(torch.mul((pred - labels).unsqueeze(1), features), dim=0)
        b -= learning_rate * torch.mean(pred - labels)

    return [round(e, 4) for e in w.tolist()], round(b.item(), 4), losses

sigmoid_neuron(features, labels, initial_weights, initial_bias, learning_rate, epochs)

([0.0989, 0.0585], -0.0841, [0.8006, 0.7631, 0.7307, 0.7025, 0.6776])

In [434]:
data_path = "/home/an/spbu_deep_learning/done"
train_x = pd.read_csv(f'{data_path}/train_x.csv', index_col=0)
train_y = pd.read_csv(f'{data_path}/train_y.csv', index_col=0)
test_x = pd.read_csv(f'{data_path}/test_x.csv')
train_y.head()

,year
133081,2001
111484,2006
448402,1989
254414,1997
272580,2001


In [260]:
new_train_y = train_y[train_y['year'].isin([2001, 1961])]
new_indices = new_train_y.index
new_train_x = train_x.loc[new_indices]

features = torch.tensor(new_train_x.values, dtype=torch.float32)
labels = torch.tensor((new_train_y['year'] == 2001).astype(int).values, dtype=torch.float32)

initial_weights = torch.randn(features.shape[1])
initial_bias = torch.zeros(1)

learning_rate = 0.001
epochs = 5

sigmoid_neuron(features, labels, initial_weights, initial_bias, learning_rate, epochs)

([0.1261,
  -0.2034,
  -1.051,
  -0.3494,
  -1.0502,
  1.2616,
  0.4137,
  -1.1475,
  2.1235,
  -0.4904,
  0.7656,
  -0.4325,
  1.7522,
  11.6982,
  8.987,
  6.5921,
  3.3813,
  5.5368,
  2.4356,
  2.9917,
  2.4621,
  1.2084,
  0.4421,
  2.4105,
  0.7774,
  -0.4157,
  -0.5242,
  0.6327,
  -0.7025,
  1.0223,
  1.0385,
  -1.4954,
  -1.2859,
  1.687,
  -0.1626,
  -0.7063,
  -0.3783,
  0.1948,
  0.8552,
  1.3663,
  -0.2249,
  -0.1879,
  -0.4951,
  1.3973,
  0.3444,
  0.2164,
  1.2698,
  0.6251,
  -1.0704,
  -0.5806,
  -2.0194,
  0.2839,
  -0.4351,
  -0.9872,
  -0.4606,
  -0.4964,
  0.4262,
  0.3206,
  -0.8494,
  0.8067,
  0.3319,
  0.6233,
  -0.1835,
  -1.4181,
  0.7365,
  -1.4411,
  0.3597,
  0.6622,
  -1.3785,
  -0.7035,
  1.4022,
  -0.8445,
  1.0985,
  0.5402,
  -1.0767,
  -0.2261,
  1.0861,
  -1.8878,
  -1.9887,
  -1.0133,
  -0.7579,
  -0.4751,
  -0.752,
  -1.0093,
  1.9194,
  -0.695,
  -0.3172,
  0.1396,
  0.2589,
  0.0392],
 0.0048,
 [43.8954, 43.8954, 43.8954, 43.8954, 43.8954])

In [402]:
features = torch.tensor([[1.0, 2.0], [2.0, 1.0], [-1.0, -2.0]]) 
labels = torch.tensor([1, 0, 0]) 
initial_weights = torch.tensor([0.1, -0.2])
initial_bias = torch.tensor(0.0) 
learning_rate = 0.1 
epochs = 5

def sigmoid_neuron_with_momentum(features, labels, initial_weights, initial_bias, learning_rate, epochs, momentum=0.9):
    w = initial_weights.clone()
    b = initial_bias.clone()
    losses = []
    eps = 1e-20
    v_w = torch.zeros_like(w)
    v_b = torch.zeros_like(b)
    for _ in range(epochs):
        pred = torch.sigmoid(w @ features.T + b)
        loss = -torch.mean(torch.mul(labels, torch.log(pred+eps)) + torch.mul((1-labels), torch.log(1-pred+eps)))
        losses.append(round(loss.item(), 4))

        w -= torch.mul(momentum, v_w) + torch.mul(learning_rate, torch.mean(torch.mul((pred - labels).unsqueeze(1), features), dim=0))
        b -= torch.mul(momentum, v_b) + torch.mul(learning_rate, torch.mean(pred - labels))

    return [round(e, 4) for e in w.tolist()], round(b.item(), 4), losses

sigmoid_neuron_with_momentum(features, labels, initial_weights, initial_bias, learning_rate, epochs)

([0.0989, 0.0585], -0.0841, [0.8006, 0.7631, 0.7307, 0.7025, 0.6776])